In [ ]:
library(xCell)
library(data.table)
library(dplyr)
library(ggplot2)
library(dplyr)
library(tidyr)

# xCell estimation

In [3]:
## filter xcell cells
cells = fread('../data/xcell/category_xcell.csv', sep='\t')
print(dim(cells))
use_types = cells[cells$Category != 'Others']$`Cell types`
print(length(use_types))

[1] 64  3
[1] 34


In [4]:
## load and merge genes
raw = fread('../data/GTEx/whole_blood/gene_tpm_2017-06-05_v8_whole_blood.gct.gz', skip=2)
common_genes = intersect(xCell.data$genes, raw$Description)
sprintf('#Common genes: %s', length(common_genes))
df = raw[raw$Description %in% xCell.data$genes]
tmp = df %>% count(Description)
use_genes = tmp[tmp$n == 1]$Description
sprintf('#Use genes: %s', length(use_genes))
df = df[df$Description %in% use_genes]
sprintf('dim(df):')
print(dim(df))

[1] "#Common genes: 10766"

[1] "#Use genes: 10752"

[1] "dim(df):"

[1] 10752   758


In [5]:
expr = as.matrix(df[,-c(1,2,3)])
rownames(expr) = df$Description

In [6]:
wrapped_scores = xCellAnalysis(expr, signatures = xCell.data$signatures, 
                               genes = use_genes, spill = xCell.data$spill,
                              rnaseq = TRUE, file.name = NULL, scale = TRUE, alpha = 0.5,
                              save.raw = FALSE, parallel.sz = 4, parallel.type = "SOCK",
                              cell.types.use = use_types)

[1] "Num. of genes: 10752"


Warning message:
“Calling gsva(expr=., gset.idx.list=., method=., ...) is deprecated; use a method-specific parameter object (see '?gsva').”


Setting parallel calculations through a MulticoreParam back-end
with workers=4 and tasks=100.
Estimating ssGSEA scores for 489 gene sets.
[1] "Calculating ranks..."
[1] "Calculating absolute values from ranks..."
  |======================================================================| 100%



In [11]:
norm_scores = t(t(wrapped_scores) / rowSums(t(wrapped_scores)))
score_df = data.frame(norm_scores)
score_df = cbind('Cell types'=rownames(score_df), score_df)
score_df = merge(score_df, cells[,c('Cell types', 'Category')], by='Cell types')
score_df <- score_df %>% relocate(Category, .after='Cell types')
props = score_df %>% group_by(Category) %>% summarise(across(starts_with('GTE'), sum))

In [13]:
props

Category,GTEX.111YS.0006.SM.5NQBE,GTEX.1122O.0005.SM.5O99J,GTEX.1128S.0005.SM.5P9HI,GTEX.113IC.0006.SM.5NQ9C,GTEX.113JC.0006.SM.5O997,GTEX.117XS.0005.SM.5PNU6,GTEX.117YW.0005.SM.5NQ8Z,GTEX.1192W.0005.SM.5NQBQ,GTEX.1192X.0005.SM.5NQC3,⋯,GTEX.ZVE2.0006.SM.51MRW,GTEX.ZVP2.0005.SM.51MRK,GTEX.ZVT2.0005.SM.57WBW,GTEX.ZVT3.0006.SM.51MT9,GTEX.ZVT4.0006.SM.57WB8,GTEX.ZVTK.0006.SM.57WBK,GTEX.ZVZP.0006.SM.51MSW,GTEX.ZVZQ.0006.SM.51MR8,GTEX.ZXES.0005.SM.57WCB,GTEX.ZXG5.0005.SM.57WCN
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
B,0.032612533,0.274504561,3.552003e-01,0.201679892,0.130089203,0.23330556,0.159588890,0.133484050,0.064906292,⋯,0.089625281,0.099321286,0.2378419499,0.108972650,0.289059213,0.291362979,1.286309e-02,0.232390586,0.365172707,0.130933917
CD4,0.156775094,0.169138308,1.459887e-01,0.288760267,0.241357286,0.23134876,0.300131447,0.212402010,0.187629586,⋯,0.272436092,0.097354068,0.1974162314,0.324088607,0.114530034,0.287615513,1.059322e-01,0.244819367,0.146861984,0.161223787
CD8,0.064318430,0.011648574,2.186784e-01,0.342325694,0.307438682,0.26313044,0.322773585,0.291441643,0.388288179,⋯,0.125597171,0.108118942,0.0139580241,0.334299922,0.144901971,0.249317024,6.009746e-18,0.310923162,0.110225449,0.378938385
DC,0.018973660,0.003892615,9.162977e-03,0.018110122,0.031737225,0.01621322,0.009801839,0.024221566,0.004142437,⋯,0.002882408,0.032123261,0.0184524762,0.012828057,0.013296213,0.011224148,3.864835e-02,0.013116641,0.009805213,0.010061781
Lymphoids,0.003296698,0.001958358,5.657640e-21,0.007256343,0.000000000,0.02019164,0.003125931,0.001630643,0.004610102,⋯,0.017369581,0.007726714,0.0002725131,0.003074945,0.000536416,0.001054683,4.813249e-04,0.005073924,0.003932418,0.004152275
Monocyte,0.331168726,0.223461473,9.407191e-02,0.071788162,0.132672094,0.04886407,0.026279279,0.095056773,0.105909278,⋯,0.174674555,0.350757595,0.2445343426,0.057638959,0.123576249,0.041647495,4.358439e-01,0.034933789,0.136051966,0.119631189
Myeloids,0.093574062,0.056242538,9.572882e-02,0.027403508,0.129669090,0.12684079,0.127652407,0.163281797,0.101506307,⋯,0.166274810,0.033992085,0.0402755194,0.081234094,0.191854608,0.067668843,6.890955e-02,0.084937358,0.038166293,0.080258570
NK,0.156621556,0.119184337,4.949953e-02,0.032876090,0.008475587,0.03863759,0.036934868,0.051918016,0.089373639,⋯,0.060950824,0.196791729,0.0985751947,0.055389756,0.058378924,0.039160041,2.102179e-01,0.042140345,0.084799017,0.099304781
Stem,0.142659243,0.139969237,3.166939e-02,0.009799922,0.018560834,0.02146793,0.013711753,0.026563502,0.053634179,⋯,0.090189278,0.073814321,0.1486737485,0.022473010,0.063866373,0.010949276,1.271037e-01,0.031664827,0.104984952,0.015495315


In [ ]:
fwrite(props, '../data/xcell/xcell_props.csv', sep='\t')